In [2]:
import sumolib
import pandas as pd
import geopandas as gpd
from shapely import wkt
import osmnx as ox
import subprocess
import folium
import openrouteservice as ors


In [3]:
net = sumolib.net.readNet(r"C:\Users\Valeria\Downloads\osmbis.net.xml.gz")

In [9]:
edge_list = net.getEdges()
node_list = net.getNodes()

print(f"The road network has {len(edge_list)} edges and {len(node_list)} nodes.")

The road network has 40204 edges and 17943 nodes.


In [4]:

outflow_df = pd.read_csv('outflow_per_region.csv')
outflow_df['zone_geometry'] = outflow_df['zone_geometry'].apply(wkt.loads)
outflow_gdf = gpd.GeoDataFrame(outflow_df, geometry='zone_geometry')

In [5]:
# cerco i centroidi della regione per cui outflow è max e centroidi di una regione centrale a caso ( 'LEUVEN-CENTRUM' in questo caso )

geom_max_out = outflow_gdf.loc[outflow_gdf['flows_car_aggr'].idxmax(), 'zone_geometry'].centroid
geom_centro = outflow_gdf.loc[outflow_gdf['SECNAAM'] == 'LEUVEN-CENTRUM', 'zone_geometry'].iloc[0].centroid
lat_centro, lon_centro = geom_max_out.y, geom_max_out.x
lat_maxflow, lon_maxflow = geom_centro.y, geom_centro.x

# convert into SUMO coordinates (x, y)
x_maxflow, y_maxflow = net.convertLonLat2XY(lon_maxflow, lat_maxflow)
x_centro, y_centro = net.convertLonLat2XY(lon_centro, lat_centro)

print(f"SUMO reference coordinates (x,y) for centrum: {x_centro,y_centro}")
print(f"SUMO reference coordinates (x,y) for maxflow: {x_maxflow,y_maxflow}")

SUMO reference coordinates (x,y) for centrum: (13118.32514550921, 8252.16544956062)
SUMO reference coordinates (x,y) for maxflow: (12883.954596722615, 9222.608083222061)


In [6]:
# the set of Neighboring edges
candiates_edges = net.getNeighboringEdges(x_maxflow, y_maxflow, r=25)
edges_and_dist_maxflow = sorted(candiates_edges, key = lambda x: x[1])

candiates_edges_centro = net.getNeighboringEdges(x_centro, y_centro, r=25)
edges_and_dist_centro = sorted(candiates_edges_centro, key = lambda x: x[1])


In [8]:
closest_edge_maxflow = edges_and_dist_maxflow[1][0]
closest_edge_centro = edges_and_dist_centro[1][0]

print(f"Name closest edge to maxflow: {closest_edge_maxflow.getName()}")
print(f"Edge ID: {closest_edge_maxflow.getID()}")

print(f"Name closest edge to centrum: {closest_edge_centro.getName()}")
print(f"Edge ID: {closest_edge_centro.getID()}")

Name closest edge to maxflow: Grote Markt
Edge ID: -510485147#11
Name closest edge to centrum: Paul Lebrunstraat
Edge ID: 3351197


In [9]:
closest_edge_maxflow, closest_edge_centro

(<edge id="-510485147#11" from="cluster_1573474888_1573474889_20910628" to="cluster_1453761232_1573474879_17886258"/>,
 <edge id="3351197" from="16526534" to="16483865"/>)

In [10]:
e_origin, e_destination = closest_edge_maxflow, closest_edge_centro

# shortest path 
shortest_path = net.getOptimalPath(e_origin, e_destination, fastest=False)
shortest_path
print(f"Number of edges in the path: {len(shortest_path[0])}")
print(f"Cost [m]: {shortest_path[1]}")


Number of edges in the path: 26
Cost [m]: 951.41


### duarouter

In [12]:

command_str_astar = "duarouter --route-files traffic_demand_leuven.rou.xml "+\
        " --net-file osm.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter_astar.rou.xml --routing-algorithm astar"

p = subprocess.Popen(command_str_astar, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

In [14]:
command_str_w = "duarouter --route-files traffic_demand_leuven.rou.xml "+\
        " --net-file osm.net.xml"+\
        " --output-file traffic_demand_pisa_duarouter_astar.rou.xml --routing-algorithm astar --weights.random-factor 300"

p = subprocess.Popen(command_str_w, shell=True, stdout=subprocess.PIPE, 
                                     stderr=subprocess.STDOUT)
retval = p.wait()

### TraCi


In [11]:
import traci
from utils import init_traci

In [12]:
# initialize TraCi by specifing the configuration file

init_traci(r"C:\Users\Valeria\Downloads\2023-11-17-09-56-17\2023-11-17-09-56-17\osm.sumocfg")

n_steps = 10

# simulate each step
for step in range(n_steps):
    
    # perform a simulation step
    traci.simulationStep()
    
    # get the list of active vehicles (vehicles inserted in the simulation)
    vehicle_list = traci.vehicle.getIDList()
    
    # value retrieval
    for v_id in vehicle_list:
        
        # some examples of value you can retrieve
        
        # Speed [m/s]
        speed_ms = traci.vehicle.getSpeed(v_id)
        
        # CO2 emissions [mg/s] 
        co2_emissions = traci.vehicle.getCO2Emission(v_id)
        
        # Position
        x, y = traci.vehicle.getPosition(v_id)
        lon, lat = traci.simulation.convertGeo(x, y)
        
        # Network info 
        road = traci.vehicle.getRoadID(v_id)

        time_loss = traci.vehicle.getTimeLoss(v_id)

        print(f"Vehicle ID: {v_id}")
        print(f"position [lon, lat]: {lon, lat}")
        print(f"speed [m/s]: {speed_ms}")
        print(f"CO2 emissions [mg/s]: {co2_emissions}\n")
        print("EdgeID of veh ",v_id , ": ",road)
        print(f"Time Loss since departure: {time_loss}")
        
        
# close TraCi when the total number of steps to simulate is reached
traci.close()

 Retrying in 1 seconds
Vehicle ID: bike0
position [lon, lat]: (4.72799510277012, 50.89708366149779)
speed [m/s]: 0.8139278191141783
CO2 emissions [mg/s]: 0.0

EdgeID of veh  bike0 :  23950551
Time Loss since departure: 0.861785842037215
Vehicle ID: bike1
position [lon, lat]: (4.7053883958980744, 50.878629217065246)
speed [m/s]: 0.0
CO2 emissions [mg/s]: 0.0

EdgeID of veh  bike1 :  -3991634#2
Time Loss since departure: 0.0
Vehicle ID: bike2
position [lon, lat]: (4.702461652372293, 50.861626760007866)
speed [m/s]: 0.0
CO2 emissions [mg/s]: 0.0

EdgeID of veh  bike2 :  3701212#0
Time Loss since departure: 0.0
Vehicle ID: bike3
position [lon, lat]: (4.70289046859455, 50.85911803905311)
speed [m/s]: 3.6895
CO2 emissions [mg/s]: 0.0

EdgeID of veh  bike3 :  838469178
Time Loss since departure: 0.0
Vehicle ID: bus0
position [lon, lat]: (4.686940977701068, 50.91253053695365)
speed [m/s]: 1.016165761789307
CO2 emissions [mg/s]: 7728.486211976913

EdgeID of veh  bus0 :  -22357190#0
Time Loss si

### OpenStreetMap routing API 


In [21]:
YOUR_KEY = '5b3ce3597851110001cf624899fee7dd897b42e19b4300590818493a'

In [1]:
import openrouteservice

In [26]:

x_origin, y_origin = e_origin.getFromNode().getCoord()
lon_origin, lat_origin = net.convertXY2LonLat(x_origin, y_origin)


x_dest, y_dest = e_destination.getFromNode().getCoord()
lon_dest, lat_dest = net.convertXY2LonLat(x_dest, y_dest)

coordinates = [[lon_origin, lat_origin], [lon_dest, lat_dest]]
client = ors.Client(key=YOUR_KEY)


route_osm = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        preference="shortest",
        format='geojson',
        geometry_simplify=False,
        validate=True)

# extract the GPS points (lon, lat) of the fastest route
gps_points_osm = [coord for coord in route_osm['features'][0]['geometry']['coordinates']]

In [59]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(coord1, coord2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = radians(coord1[1]), radians(coord1[0])
    lat2, lon2 = radians(coord2[1]), radians(coord2[0])

    # Calculate the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = R * c

    return distance

# Example usage
coord1 = gps_points_osm[0]
coord2 = gps_points_osm[-1]

distance_km = haversine_distance(coord1, coord2)
print(f"The distance between the two points is {distance_km:.2f} kilometers.")


The distance between the two points is 1.14 kilometers.


In [30]:

m = folium.Map(location=[lat_origin, lon_origin], tiles='cartodbpositron', zoom_start=13)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in gps_points_osm], popup = 'shortest').add_to(m)
m

In [31]:
x_origin, y_origin = e_origin.getFromNode().getCoord()
lon_origin, lat_origin = net.convertXY2LonLat(x_origin, y_origin)


x_dest, y_dest = e_destination.getFromNode().getCoord()
lon_dest, lat_dest = net.convertXY2LonLat(x_dest, y_dest)

coordinates = [[lon_origin, lat_origin], [lon_dest, lat_dest]]
client = ors.Client(key=YOUR_KEY)


route_osm = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        preference="recommended",
        format='geojson',
        geometry_simplify=False,
        validate=True)

# extract the GPS points (lon, lat) of the fastest route
gps_points_osm_recom = [coord for coord in route_osm['features'][0]['geometry']['coordinates']]


folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in gps_points_osm_recom], color='red', popup='recommended').add_to(m)
m